In [2]:
import branches
import metrics
import plotting
import preprocess
import segmentation
import thresholding
import file

import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import skimage.morphology as morphology

In [3]:
parent_directory = os.path.split(file.current_directory)[0]
datapath = parent_directory + "\\training\\0520\\"
plotpath = parent_directory + "\\plots\\"

In [4]:
RNA = pd.read_csv(datapath + "barcodes.csv")

In [5]:
def save(img,name,dir):
    # saveim = Image.fromarray(img)
    # saveim.save(dir+name)
    np.save(dir+name+".npy",img)

### Create process and body image for dataset:

In [6]:
from skimage.measure import label

processImageFile = datapath + "processImage.npy"
somaImageFile = datapath + "somaImage.npy"

if not (os.path.exists(processImageFile) and os.path.exists(somaImageFile)):

        imname = 'preprocessed_Probabilities.png'

        datafile = datapath + imname
        segProbIm = cv2.imread(datafile)

        print("thresholding image")
        thresholdedProcessImage, somaImage = thresholding.threshold_img(segProbIm)
        print("breaking down image")
        brokenProcessImage, togetherskel = segmentation.break_down(thresholdedProcessImage)

        cleanedProcessImage = morphology.remove_small_objects(brokenProcessImage>0,min_size=10,connectivity=1)
        cleanedSomaImage = morphology.remove_small_objects(somaImage>0,min_size=25,connectivity=1)

        processImage = label(cleanedProcessImage,connectivity=1)
        somaImage = label(cleanedSomaImage,connectivity=2)

        save(processImage, "processImage", datapath)
        save(somaImage, "somaImage", datapath)

processImage = np.load(processImageFile)
somaImage = np.load(somaImageFile)

### Build process and soma objects

In [7]:
processes = branches.buildProcesses(processImage, RNA)
somas = branches.buildSomas(somaImage, RNA)

100%|██████████| 142/142 [00:01<00:00, 100.26it/s]


In [8]:
bbs = [[300,1000,500,500],[2500,2700,500,500]]

baseimgfile = datapath+"preprocessed.png"
baseimg = cv2.imread(baseimgfile)

probimname = 'preprocessed_Probabilities.png'
datafile = datapath + probimname
segProbIm = cv2.imread(datafile)

# plotting.embedimg(baseimg, bbs, plotpath)
# plotting.embedimg(segProbIm, bbs, datapath, name="probabilities")
# plotting.embedimg(processImage, bbs, datapath, name="processImg")

In [9]:
for process in list(processes.values())[:20]:
        neighbors = process.getNeighbors(somaImage, 'soma', 10)

In [10]:
# import importlib
# importlib.reload(PyMC3)

In [11]:
branch_list = []
for process in list(processes.values())[:20]:
        for soma in process.neighbors['soma']:
                root = branches.Root(somas[soma])
                branch_list.append(branches.Branch(root, process))
print(branch_list)

[<branches.Branch object at 0x000002B594050DF0>, <branches.Branch object at 0x000002B58EF646A0>, <branches.Branch object at 0x000002B58EF7D610>, <branches.Branch object at 0x000002B58EF7DFD0>, <branches.Branch object at 0x000002B594050D00>, <branches.Branch object at 0x000002B5940505B0>, <branches.Branch object at 0x000002B594050EE0>]


C:\Lab Work\segmentation\Segmentation\branches.py:168: RuntimeWarning: invalid value encountered in multiply
  disIm = (1 - self.im)*np.inf


In [12]:
for i, branch in enumerate(branch_list[:5]):
        rnaids, rnadists = branch.get_rnadistances(True)
        print(rnadists)
        # plotting.plotKDE(rnadists, "RNA distribution along candidate branch", savefile=datapath+f'rnabranch{i}.png')

[ 52.1626118  26.1246118  76.8186118 110.1306118 112.9586118  61.3026118
 133.6146118  57.2326118  59.2326118 115.5446118  48.3346118 129.0286118
  25.9526118  40.8506118  61.0606118 110.1306118  82.2006118  74.1306118
  70.9586118  48.3346118  51.9906118  59.7486118  96.3026118  93.5126118
  86.0286118  62.7166118  84.5446118  91.8886118  99.7166118 115.9586118
 130.6146118  83.0606118  58.6466118  61.7486118  25.3666118  93.9266118
  92.5126118  65.0606118  43.2646118 135.6146118  23.9526118  70.9906118
  95.8886118  66.4746118  93.5126118  46.6086118  27.9526118  59.5066118
  50.7486118  23.9526118 137.2006118 131.7866118  36.1946118]
[ 6.828 10.656  7.242  6.828  9.242 11.898  9.656 11.242  8.242 10.07
 10.484 11.242 11.242 10.242 10.07   7.242  9.242 10.898  5.242  5.828
 10.07   6.828  9.242  6.242  9.656 10.484 10.242 10.07   6.242  7.828
  9.656  9.242  9.656 11.242  7.242  3.828  9.828  5.242]
[ 9.47213595 20.54213595 15.71413595 10.88613595 14.88613595]
[6. 6.]
[ 42.14   45.3

In [13]:
for branch in branch_list[:5]:
        process = branch.end
        soma = branch.source.end
        # print(process.ymin, process.xmin, process.label, process.neighbors)
        # print(soma.ymin, soma.xmin, soma.label)
        # plotting.implot(1*(processImage[branch.ymin:branch.ymax,branch.xmin:branch.xmax]==process.label) + (somaImage[branch.ymin:branch.ymax,branch.xmin:branch.xmax]==soma.label)*2, datapath, show=False)
        # plotting.implot((somaImage[branch.ymin:branch.ymax,branch.xmin:branch.xmax]==soma.label)*2, datapath, show=False)
        # plt.show()

In [1]:
import pymc3 as pm

branch = branch_list[4]

with pm.Model() as experiment:
        # beta = pm.Uniform('beta',0,1)
        # sigma = pm.Bernoulli('sigma',p=beta)
        lam = pm.HalfNormal('lam', sd = 5)
        # x = pm.Uniform('x',np.min(branch.rnadists),np.max(branch.rnadists))
        # distance = (2*sigma-1)*pm.Exponential('delta',lam) + x
        y = pm.Exponential('y',lam,observed=branch.rnadists)

        trace = pm.sample(10)
        pm.plot_trace(trace)
        plt.show()


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


NameError: name 'branch_list' is not defined